In [ ]:
import utils.ss_transforms as sstr
from steps import Args,createServerStep1,createServerStep2,createServerStep3,createServerStep4,createServerStep5clustering
from utils.FDA import extractClientsStyles,applyFDAstyles

In [ ]:
#STEP 1 
arg=Args(num_rounds=8,num_epochs=25,bs=4,testEachRounds=1)
trainTr=sstr.Compose([sstr.RandomCrop(scale=(0.5,1),ratio=(0.9,1.1)),sstr.Resize((540,960)),sstr.ToTensor()])
testTr=sstr.Compose([sstr.Resize((540,960)),sstr.ToTensor()])
server=createServerStep1(args=arg,train_transform=trainTr,test_transform=testTr)
server.train()

In [ ]:
#STEP 2
arg=Args(clients_per_round=2,num_epochs=3,num_rounds=1200,bs=4,testEachRounds=1)
trainTr=sstr.Compose([sstr.RandomCrop(scale=(0.5,1),ratio=(0.9,1.1)),sstr.Resize((540,960)),sstr.ToTensor()])
testTr=sstr.Compose([sstr.Resize((540,960)),sstr.ToTensor()])
server=createServerStep2(args=arg,train_transform=trainTr,test_transform=testTr)
server.train()

In [ ]:
#STEP 3 (with FDA)
arg=Args(num_epochs=10,num_rounds=20,bs=4,testEachRounds=1,saveEachRounds=1)
styles=extractClientsStyles(5)
trainTr=sstr.Compose([sstr.Resize((540,960)),sstr.ToTensor(),applyFDAstyles(styles)])
testTr=sstr.Compose([sstr.Resize((540,960)),sstr.ToTensor()])
server=createServerStep3(args=arg,train_transform=trainTr,test_transform=testTr)
server.train()

In [ ]:
#STEP 4
arg=Args(clients_per_round=2,num_epochs=1,num_rounds=120,bs=4,testEachRounds=1,teacher_update=None,unsupervised=True)
styles=extractClientsStyles(5)
trainTr=sstr.Compose([sstr.Resize((540,960)),sstr.ToTensor()])
testTr=sstr.Compose([sstr.Resize((540,960)),sstr.ToTensor()])
server=createServerStep4(args=arg,train_transform=trainTr,test_transform=testTr)
server.loadModel('BestModelStep3.pt')
server.eval_train()
server.test()
server.train()

In [ ]:
#STEP 5 CLUSTERING
arg=Args(clients_per_round=2,num_epochs=3,num_rounds=1200,bs=4,testEachRounds=1,clustering=True,cluster_param='FDA',num_clusters=[2,3,4,5,6,7,8])
trainTr=sstr.Compose([sstr.RandomCrop(scale=(0.5,1),ratio=(0.9,1.1)),sstr.Resize((540,960)),sstr.ToTensor()])
testTr=sstr.Compose([sstr.Resize((540,960)),sstr.ToTensor()])
server=createServerStep5clustering(args=arg,train_transform=trainTr,test_transform=testTr)
server.train()

In [ ]:
#Losses
arg=Args(num_epochs=25,rounds=8,reduction='meanClasses')
arg=Args(num_epochs=25,rounds=8,reduction='frequencyClass',reductionParam=0.1)
arg=Args(num_epochs=25,rounds=8,reduction='weightedMean',reductionParam=[1,1,1,1,1,2,2,2,1,1,1,2,2,1,1,1])